# Working with Cloud (Remote) Back-ends
## So far
- We've learnt to create a synthetic dataset
- Train a logical function to make predictions from the synthetic dataset
- Test and develop an API server with Flask
- Deploy our prediction function to Heroku

## In this section
We will learn to interact with Google's Firebse and create a document collection **to demonstrate the usecase when our application needs to collect and store user data**.

## Setup and Authentication

In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

#### Point to Firebase credentials

In [2]:
cred = credentials.Certificate("../../configs/app_creds.json")

#### Initialize firebase client as admin

In [3]:
firebase_admin.initialize_app(cred,
    {
        "databaseURL": "https://save-nyc-demo.firebaseio.com/"
})

## Interfacing with your Firebase Collections

#### Load data to bootstrap

In [4]:
df = pd.read_csv("../../data/superhero_timesheets.csv")
df

,superhero,date,active_on_duty
0,Daredevil,2022-05-29,1
1,Daredevil,2022-05-30,1
2,Daredevil,2022-05-31,1
3,Deadpool,2022-05-30,1
4,Deadpool,2022-05-31,1
5,Dr X,2022-05-29,0
6,Dr X,2022-05-30,1
7,Luke Cage,2022-05-30,0
8,Luke Cage,2022-05-31,0
9,Mr Fantastic,2022-05-31,1


#### Connect to your desired collection

In [5]:
db = firestore.client()
doc_ref = db.collection("superhero_timesheets")

#### Bootstrap all records

In [6]:
%%time
tmp = df.to_dict(orient="records")

list(map(lambda x: doc_ref.add(x), tmp))

CPU times: user 143 ms, sys: 102 ms, total: 245 ms
Wall time: 8.84 s


[(DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 30, 552103, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x129eef610>),
 (DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 30, 802361, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x129eef2b0>),
 (DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 31, 329713, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x129eef9d0>),
 (DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 31, 680735, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x129eefac0>),
 (DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 32, 212026, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x129eefd90>),
 (DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 32, 580143, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x129eef940>),
 (DatetimeWithNanoseco

#### Query to fetch all records for today

In [7]:
today = pd.Timestamp.now().date()
res_0 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_0

,active_on_duty,date,superhero
0,1,2022-05-31,Spiderman
1,1,2022-05-31,Deadpool
2,1,2022-05-31,Mr Fantastic
3,0,2022-05-31,Luke Cage
4,1,2022-05-31,Daredevil
5,1,2022-05-31,Wolverine
6,1,2022-05-31,Peeping Tom


#### Adding a record

In [8]:
%%time
doc_ref.add({
    "superhero": "Leonardo",
    "active_on_duty": 1,
    "date": "2022-05-31"
})

CPU times: user 3.52 ms, sys: 6.28 ms, total: 9.79 ms
Wall time: 520 ms


(DatetimeWithNanoseconds(2022, 5, 31, 9, 26, 38, 396862, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x129e82940>)

In [9]:
res_1 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_1

,superhero,date,active_on_duty
0,Spiderman,2022-05-31,1
1,Deadpool,2022-05-31,1
2,Leonardo,2022-05-31,1
3,Mr Fantastic,2022-05-31,1
4,Luke Cage,2022-05-31,0
5,Daredevil,2022-05-31,1
6,Wolverine,2022-05-31,1
7,Peeping Tom,2022-05-31,1


#### Dropping a collection

In [10]:
%%time
def delete_collection(coll_ref, batch_size=200):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)
    
delete_collection(doc_ref)

Deleting doc 4HMEnEzwKPc6hDL9LITM => {'active_on_duty': 1, 'superhero': 'Dr X', 'date': '2022-05-30'}
Deleting doc 4iOfwQufWPOnPgz1wB9N => {'active_on_duty': 0, 'date': '2022-05-29', 'superhero': 'Peeping Tom'}
Deleting doc 5XB4NqhrkmlFMeXPPZFN => {'date': '2022-05-29', 'active_on_duty': 0, 'superhero': 'Dr X'}
Deleting doc 6VE8RYyFfgHKIYorugfa => {'date': '2022-05-31', 'superhero': 'Spiderman', 'active_on_duty': 1}
Deleting doc 9CENdrHjDFo9wMEcJKDL => {'active_on_duty': 1, 'date': '2022-05-31', 'superhero': 'Deadpool'}
Deleting doc FrTjsUKUVZPF1sCRjIN0 => {'active_on_duty': 1, 'superhero': 'Leonardo', 'date': '2022-05-31'}
Deleting doc HM2RP1WqC62a5JxGv8kf => {'date': '2022-05-30', 'active_on_duty': 1, 'superhero': 'Spiderman'}
Deleting doc HyWYswe2JoPm7aqfK2Ck => {'superhero': 'Mr Fantastic', 'date': '2022-05-31', 'active_on_duty': 1}
Deleting doc KFMAWcd9xIpmXrv30rwF => {'active_on_duty': 1, 'date': '2022-05-29', 'superhero': 'Daredevil'}
Deleting doc QbwgjYa6NhxTmVEBhTbP => {'date'

## Outro
Thank you for following this workshop all the way to the ends. All credits to [Firebase Docs](https://firebase.google.com/docs) for code snippets used in this notebook.